<a href="https://colab.research.google.com/github/krystaldowling/MAST30034_Final_Project-/blob/machine_learning_doc2vec/classifier_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import scipy
import pickle

from sklearn.model_selection import train_test_split

news_data = pd.read_csv("final_preproccessed_data.csv")

news_data.index.name = 'index'

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from gensim.test.utils import get_tmpfile

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

fname = get_tmpfile("my_doc2vec_model")


model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)


def doc2vec_fun(x):
    L = []
    for elem in x.split(' '):
        elem = (elem.replace('\n', '').replace(',', '').replace('.', '').
        replace('”', '').replace('?', '').replace('\'', '').lower().
        replace('“', '').replace(']', '').replace('[', ''))
        L.append(elem)
    vector = model.infer_vector(L)
    return vector
news_data['doc2vec_title']=news_data['title'].map(lambda x:doc2vec_fun(x))
news_data['doc2vec_text']=news_data['text'].map(lambda x:doc2vec_fun(x))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[-0.03556247 -0.09903148  0.07664264  0.09468351 -0.02956764]
                                                   title  ...                                       doc2vec_text
index                                                     ...                                                   
20183  Policy Shift Helps Coal, but Other Forces May ...  ...  [0.0734534, 0.0027341133, -0.01355525, -0.0641...
1750   Dr. David Duke and Adrian Salbuch : The intern...  ...  [0.03029924, -0.034626447, -0.008349692, -0.03...
1710   Donald Trump Jr. Met With Russian Advocates Ab...  ...  [-0.03895306, -0.094609015, -0.0064553455, 0.0...
3801             Why Trump Won and Why Clinton Lost    :  ...  [-0.06605714, -0.04280777, 0.03766791, -0.0653...
24095  The Battle of ‘Miss Saigon’: Yellowface, Art a...  ...  [0.053424653, 0.038532283, 0.012829526, 0.0366...
23109  Trump Meets With Some Vocal Critics: Editors a...  ...  [0.026124837, -0.029834874, -0.08105983, -0.09...
12983  YouTube Advertiser Exodus H

In [ ]:
# this is only the text data which has been converted to a data frame.
# each row is one article and each column is an element of the vector. 

text_vectorised_data = pd.DataFrame(news_data['doc2vec_text'].array, columns = range(100))
title_vectorised_data = pd.DataFrame(news_data['doc2vec_title'].array, columns = range(100))

# once we have vectorised the entire datset then split into train and test! 
text_vec_train, text_vec_test, label_train, label_test = train_test_split(text_vectorised_data, news_data['label'], test_size=0.33, 
                                                        random_state=88)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
# trying different SVM kernels

def svm_classifier(X, y):
    
    models = [SVC(kernel = 'linear'),
             SVC(kernel = 'poly'),
             SVC(kernel = 'rbf'),
             SVC(kernel = 'sigmoid')]
    
    titles = ['SVC with linear kernel',
            'SVC with polynomial kernel',
             'SVC with rbf kernel',
             'SVC with sigmoid kernel']


    
    # prepare the cross-validation procedure
    cv = KFold(n_splits=4, random_state=1, shuffle=True)
    
    for title, model in zip(titles, models):
        print('\n',title, ': with doc2vec')
        model = model
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
        print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
            
    return

svm_classifier(text_vectorised_data, news_data['label'])

In [ ]:
# trying different SVM kernels

def logreg_classifier(X, y):
    

    models = [LogisticRegression(solver = 'newton-cg'),
             LogisticRegression(solver = 'lbfgs'),
             LogisticRegression(solver = 'liblinear'),
             LogisticRegression(solver = 'sag'),
             LogisticRegression(solver = 'saga')]
    
    titles = ['Logistic Regression with newton-cg',
            'Logistic Regression with lbfgs',
             'Logistic Regression with liblinear',
             'Logistic Regression with sag',
             'Logistic Regression with saga']


    
    # prepare the cross-validation procedure
    cv = KFold(n_splits=4, random_state=1, shuffle=True)
    
    for title, model in zip(titles, models):
        print('\n',title, ': doc2vec')
        model = model
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
        print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
            
    return

logreg_classifier(text_vectorised_data, news_data['label'])

ValueError: ignored